About the approximation properties of P1 CG FEM
========================

In [ ]:
from dune.xt.common import init_mpi
init_mpi()

In [ ]:
from collections import OrderedDict
from dune.xt.grid import (make_cube_grid__1d_yaspgrid as make_cube_grid,
                          make_boundary_info_on_leaf_layer as make_boundary_info)

def make_grid(n):
    return make_cube_grid(lower_left=[-1], upper_right=[1], num_elements=[n])

grids = OrderedDict()
for num_elements in (2, 4, 8, 128):
    print('creating grid with {} elements ...'.format(num_elements))
    grids[num_elements] = make_grid(num_elements)
max_elements = list(grids.keys())[-1]

boundary_info = make_boundary_info(grids[max_elements], 'xt.grid.boundaryinfo.alldirichlet')

In [ ]:
from dune.xt.functions import make_constant_function_1x1, make_expression_function_1x1

diffusion = make_constant_function_1x1(grids[max_elements], 1.0, name='diffusion')
#diffusion = make_expression_function_1x1(grids[max_elements], 'x', '0.71*(1+(cos(2*pi*(x[0]/0.1))*cos(2*pi*(x[0]/0.1))))', order=3, name='diffusion')
force = make_constant_function_1x1(grids[max_elements], 1.0, name='force')
diffusion.visualize(grids[max_elements], 'diffusion')
exact_solution = make_expression_function_1x1(grids[max_elements], 'x', '-0.5*x[0]*x[0] + 0.5', order=2, name='exact solution')
exact_solution.visualize(grids[max_elements], 'exact_solution')

In [ ]:
from dune.gdt import (make_cg_leaf_part_to_1x1_fem_p1_space as make_space,
                      make_discrete_function_istl_dense_vector_double,
                      project)                      

def compute_best_approximation(grid, filename_suffix):
    space = make_space(grid)
    best_approximation = make_discrete_function_istl_dense_vector_double(space, name='best approximation')
    project(exact_solution, best_approximation)
    best_approximation.visualize('best_approximation_' + filename_suffix)

In [ ]:
for num_elements, grid in grids.items():
    compute_best_approximation(grid, '{}'.format(num_elements).zfill(len(str(max_elements))))

In [ ]:
from dune.xt.la import IstlDenseVectorDouble, make_solver
from dune.gdt import (make_elliptic_matrix_operator_istl_row_major_sparse_matrix_double as make_elliptic_matrix_operator,
                      make_l2_volume_vector_functional_istl_dense_vector_double as make_l2_volume_vector_functional,
                      make_dirichlet_constraints,
                      make_discrete_function,
                      make_system_assembler)

def compute_solution(grid, filename):
    
    space = make_space(grid)
    
    elliptic_operator = make_elliptic_matrix_operator(diffusion, space)
    l2_force_functional = make_l2_volume_vector_functional(force, space)
    dirichlet_constraints = make_dirichlet_constraints(boundary_info, space.size(), True)
    
    system_assembler = make_system_assembler(space)
    system_assembler.append(elliptic_operator)
    system_assembler.append(l2_force_functional)
    system_assembler.append(dirichlet_constraints)
    system_assembler.assemble()
    
    lhs_matrix = elliptic_operator.matrix()
    rhs_vector = l2_force_functional.vector()
    dirichlet_constraints.apply(lhs_matrix, rhs_vector)
    
    solution = IstlDenseVectorDouble(space.size(), 0.)
    make_solver(lhs_matrix).apply(rhs_vector, solution)
    
    make_discrete_function(space, solution, 'discrete solution').visualize(filename)

In [ ]:
for num_elements, grid in grids.items():
    compute_solution(grid, 'discrete_solution_' + '{}'.format(num_elements).zfill(len(str(max_elements))))